In [1]:
import os
import numpy as np
import cv2
from PIL import Image
from Anisotropic import anisodiff
import random
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

In [2]:
def convert_images_in_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(input_folder, filename)
            img = Image.open(img_path)
            bw_img = np.array(img.convert('L'))
            filtered_image = anisodiff(bw_img)
            output_path = os.path.join(output_folder, filename)
            Image.fromarray(filtered_image.astype(np.uint8)).save(output_path)

In [3]:
def extract_random_windows(original_img, filtered_img, k, N):
    windows = []
    img_height, img_width = original_img.shape
    
    for _ in range(N):
        x = random.randint(0, img_width - k)
        y = random.randint(0, img_height - k)
        
        xi = original_img[y:y+k, x:x+k]
        yi = filtered_img[y:y+k, x:x+k]
        
        windows.append((xi, yi))
    
    return windows

In [4]:
def create_dataset(input_folder, output_folder, k, N, train_ratio=0.7, val_ratio=0.15):
    windows = []
    num_images = len([name for name in os.listdir(input_folder) if name.endswith(".jpg") or name.endswith(".png")])

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            original_img_path = os.path.join(input_folder, filename)
            filtered_img_path = os.path.join(output_folder, filename)
            
            original_img = np.array(Image.open(original_img_path).convert('L'))
            filtered_img = np.array(Image.open(filtered_img_path))
            
            windows.extend(extract_random_windows(original_img, filtered_img, k, N // num_images))

    random.shuffle(windows)

    train_size = int(len(windows) * train_ratio)
    val_size = int(len(windows) * val_ratio)
    
    train_set = windows[:train_size]
    val_set = windows[train_size:train_size + val_size]
    test_set = windows[train_size + val_size:]
    
    return train_set, val_set, test_set


In [5]:
def unet_model(input_size=(32, 32, 1)):
    inputs = layers.Input(input_size)

    # Down-sampling path
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)

    # Up-sampling path
    u5 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

In [6]:
def prepare_data(data_set, k):
    X = np.array([xi.reshape((k, k, 1)) for xi, _ in data_set])
    y = np.array([yi.reshape((k, k, 1)) for _, yi in data_set])
    return X, y

input_folder_path = "images"
output_folder_path = "bw-images"
k = 32
N = 500000

convert_images_in_folder(input_folder_path, output_folder_path)
train_set, val_set, test_set = create_dataset(input_folder_path, output_folder_path, k, N)

train_X, train_y = prepare_data(train_set, k)
val_X, val_y = prepare_data(val_set, k)
test_X, test_y = prepare_data(test_set, k)

model = unet_model(input_size=(k, k, 1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)

history = model.fit(train_X, train_y, epochs=20, batch_size=64, validation_data=(val_X, val_y), callbacks=[early_stopping, reduce_lr])

test_loss, test_acc = model.evaluate(test_X, test_y)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')

Epoch 1/20
5469/5469 [==============================] - 1309s 239ms/step - loss: nan - accuracy: 4.0246e-04 - val_loss: nan - val_accuracy: 4.0596e-04 - lr: 0.0010
Epoch 2/20
5469/5469 [==============================] - 1347s 246ms/step - loss: nan - accuracy: 4.0457e-04 - val_loss: nan - val_accuracy: 4.0596e-04 - lr: 0.0010
Epoch 3/20
2344/2344 [==============================] - 103s 44ms/step - loss: nan - accuracy: 4.0221e-04
Test Loss: nan, Test Accuracy: 0.00040221353992819786


In [9]:
def inference_on_image(model, image, k):
    img_height, img_width = image.shape
    padded_image = np.pad(image, ((0, k - img_height % k), (0, k - img_width % k)), 'reflect')
    output_image = np.zeros_like(padded_image, dtype=np.float32)  # Change dtype to float32

    for y in range(0, img_height, k):
        for x in range(0, img_width, k):
            window = padded_image[y:y+k, x:x+k]
            window_input = window.reshape((1, k, k, 1))
            window_output = model.predict(window_input)
            output_image[y:y+k, x:x+k] += window_output.reshape((k, k))

    output_image = output_image[:img_height, :img_width]
    return output_image

# Example usage for inference on a test image
test_img_path = os.path.join(output_folder_path, '2018.jpg')
test_img = np.array(Image.open(test_img_path).convert('L'))
filtered_test_img = inference_on_image(model, test_img, k)

# Convert output image to uint8 before saving
filtered_test_img = Image.fromarray(np.clip(filtered_test_img, 0, 255).astype(np.uint8))
filtered_test_img.save('filtered_test_image.jpg')


1/1 [==============================] - 0s 15ms/step


/var/folders/8f/4tqfct110qxbk9qrh28g1lpr0000gn/T/ipykernel_63802/279597084.py:22: RuntimeWarning: invalid value encountered in cast
  filtered_test_img = Image.fromarray(np.clip(filtered_test_img, 0, 255).astype(np.uint8))
